<a href="https://colab.research.google.com/github/hemil19/nlp-projects/blob/master/Kaggle_Fake_News_Classifier/Kaggle_Fake_News_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [ ]:
!pip3 install kaggle

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/kaggle"

In [3]:
%cd /content/gdrive/My Drive/kaggle

/content/gdrive/My Drive/kaggle


In [ ]:
!kaggle competitions download -c fake-news

 64% 6.00M/9.42M [00:00<00:00, 60.1MB/s]
100% 9.42M/9.42M [00:00<00:00, 59.7MB/s]
  0% 0.00/40.6k [00:00<?, ?B/s]
100% 40.6k/40.6k [00:00<00:00, 5.69MB/s]
 97% 36.0M/37.0M [00:00<00:00, 49.9MB/s]
100% 37.0M/37.0M [00:00<00:00, 63.0MB/s]


In [ ]:
#unzipping the zip files and deleting the zip files
!unzip \*.zip  && rm *.zip

Archive:  test.csv.zip
  inflating: test.csv                

Archive:  train.csv.zip
  inflating: train.csv               

2 archives were successfully processed.


In [4]:
!ls

catboost_info  kaggle.json  submit.csv	test.csv  train.csv


In [5]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np

In [6]:
df=pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
test=pd.read_csv('test.csv')
test.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


#Removing NA VALUES

In [8]:
df.isna().sum(),test.isna().sum()

(id           0
 title      558
 author    1957
 text        39
 label        0
 dtype: int64, id          0
 title     122
 author    503
 text        7
 dtype: int64)

In [9]:
df=df.dropna()
test['title'].fillna(test['title'].mode()[0],inplace= True)
#test=test.dropna()

In [10]:
df.isna().sum(),test.isna().sum()

(id        0
 title     0
 author    0
 text      0
 label     0
 dtype: int64, id          0
 title       0
 author    503
 text        7
 dtype: int64)

In [11]:
## Get Features
X=df.drop('label',axis=1)
y=df['label']

In [12]:
print(X.shape,y.shape)

(18285, 4) (18285,)


In [13]:
import tensorflow as tf
tf.__version__

'2.3.0'

In [14]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [15]:
### Vocabulary size
voc_size=5000

#Onehot Representation

In [16]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
messages_train=X.copy()
messages_train.reset_index(inplace=True)

messages_test=test.copy()
messages_test.reset_index(inplace=True)

#Data Preprocessing

In [18]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
traincorpus = []
for i in range(0, len(messages_train)):
  review = re.sub('[^a-zA-Z]', ' ', messages_train['title'][i])
  review = review.lower()
  review = review.split()
    
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  traincorpus.append(review)

In [19]:
testcorpus = []
for i in range(0, len(messages_test)):
  review = re.sub('[^a-zA-Z]', ' ', messages_test['title'][i])
  review = review.lower()
  review = review.split()
    
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  testcorpus.append(review)

In [20]:
traincorpus[:10],testcorpus[:10]

(['hous dem aid even see comey letter jason chaffetz tweet',
  'flynn hillari clinton big woman campu breitbart',
  'truth might get fire',
  'civilian kill singl us airstrik identifi',
  'iranian woman jail fiction unpublish stori woman stone death adulteri',
  'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
  'beno hamon win french socialist parti presidenti nomin new york time',
  'back channel plan ukrain russia courtesi trump associ new york time',
  'obama organ action partner soro link indivis disrupt trump agenda',
  'bbc comedi sketch real housew isi caus outrag'],
 ['specter trump loosen tongu purs string silicon valley new york time',
  'russian warship readi strike terrorist near aleppo',
  'nodapl nativ american leader vow stay winter file lawsuit polic',
  'tim tebow attempt anoth comeback time basebal new york time',
  'keiser report meme war e',
  'trump usa antiqu hero clinton next presid',
  'pelosi call fbi investi

#OneHotRepresentation

In [21]:
onehot_repr_train=[one_hot(words,voc_size) for words in traincorpus]
onehot_repr_test=[one_hot(words,voc_size) for words in testcorpus]
onehot_repr_train,onehot_repr_test

([[4272, 1088, 1642, 1958, 4421, 2844, 351, 3179, 1290, 3248],
  [231, 3648, 52, 2989, 3246, 1192, 3570],
  [4361, 420, 2879, 4578],
  [2042, 193, 4192, 3911, 2324, 734],
  [2705, 3246, 283, 2924, 1812, 4877, 3246, 2438, 1394, 693],
  [4740,
   4447,
   961,
   788,
   102,
   674,
   1490,
   1320,
   1129,
   274,
   3307,
   4925,
   814,
   142,
   3570],
  [4416, 3042, 145, 4219, 1447, 3487, 4252, 872, 3596, 1235, 4563],
  [4252, 4006, 3799, 4727, 3629, 1912, 674, 1605, 3596, 1235, 4563],
  [764, 3292, 4719, 509, 289, 4384, 95, 2942, 674, 4920],
  [1294, 4685, 1822, 2724, 2869, 3862, 780, 746],
  [4778, 957, 3721, 985, 4823, 371, 2066, 156, 3873, 3919, 340],
  [3911, 3125, 4421, 4384, 674, 3629],
  [1194, 3863, 343, 2403, 728, 394, 4595, 984, 2517],
  [3932, 2707, 4642, 4301, 4139, 3764, 4452, 3596, 1235, 4563],
  [1189, 1132, 4332, 761, 4849, 3596, 1235, 4563],
  [3907, 2094, 1961, 513, 3045, 1170, 4794, 540, 126, 1152],
  [1697, 1182, 3648],
  [4627, 4839, 3233, 369, 674, 2154, 

#Embedding Representation

In [22]:
sent_length=15
embedded_docs_train=pad_sequences(onehot_repr_train,padding='pre',maxlen=sent_length)
embedded_docs_test=pad_sequences(onehot_repr_test,padding='pre',maxlen=sent_length)
print(embedded_docs_train[:5],embedded_docs_test[:5])

[[   0    0    0    0    0 4272 1088 1642 1958 4421 2844  351 3179 1290
  3248]
 [   0    0    0    0    0    0    0    0  231 3648   52 2989 3246 1192
  3570]
 [   0    0    0    0    0    0    0    0    0    0    0 4361  420 2879
  4578]
 [   0    0    0    0    0    0    0    0    0 2042  193 4192 3911 2324
   734]
 [   0    0    0    0    0 2705 3246  283 2924 1812 4877 3246 2438 1394
   693]] [[   0    0    0    0  646  674 4456  378 4630 4000 4102 4547 3596 1235
  4563]
 [   0    0    0    0    0    0    0    0 4778 4324  120 2198  969  713
  1983]
 [   0    0    0    0    0 3716 3939 2087 4130 2244 1383  882 2984 1103
  2535]
 [   0    0    0    0    0 4615  966 1147  911 1591 4563 4226 3596 1235
  4563]
 [   0    0    0    0    0    0    0    0    0    0 4471 4363 4552 3208
  1288]]


In [29]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 15, 40)            200000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [30]:
len(embedded_docs_train),y.shape

(18285, (18285,))

In [31]:
import numpy as np
X_final=np.array(embedded_docs_train)
y_final=np.array(y)
test_final=np.array(embedded_docs_test)
print(X_final.shape,y_final.shape,test_final.shape)

(18285, 15) (18285,) (5200, 15)


In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.20, random_state=42)

In [33]:
print("Training Set Shape",(X_train.shape,y_train.shape))
print("Testing Set Shape",(X_test.shape,y_test.shape))

Training Set Shape ((14628, 15), (14628,))
Testing Set Shape ((3657, 15), (3657,))


#Model Training

In [34]:
### Finally Training
es=tf.keras.callbacks.EarlyStopping(patience=3,monitor='val_loss',restore_best_weights=True)
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20,batch_size=64,callbacks=[es])

Epoch 1/20
229/229 [==============================] - 6s 24ms/step - loss: 0.3012 - accuracy: 0.8579 - val_loss: 0.1963 - val_accuracy: 0.9150
Epoch 2/20
229/229 [==============================] - 5s 23ms/step - loss: 0.1328 - accuracy: 0.9478 - val_loss: 0.1991 - val_accuracy: 0.9202
Epoch 3/20
229/229 [==============================] - 5s 23ms/step - loss: 0.0976 - accuracy: 0.9622 - val_loss: 0.2227 - val_accuracy: 0.9128
Epoch 4/20
229/229 [==============================] - 5s 22ms/step - loss: 0.0690 - accuracy: 0.9753 - val_loss: 0.2560 - val_accuracy: 0.9139


#Evaluating Model Performance

In [35]:
y_pred=model.predict_classes(X_test)
from sklearn.metrics import confusion_matrix,classification_report
print("Confusion Matrix is: \n",confusion_matrix(y_test,y_pred))
print("\n")
print("Classification Report is: \n",classification_report(y_test,y_pred))

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
Confusion Matrix is: 
 [[1876  206]
 [ 105 1470]]


Classification Report is: 
               precision    recall  f1-score   support

           0       0.95      0.90      0.92      2082
           1       0.88      0.93      0.90      1575

    accuracy                           0.91      3657
   macro avg       0.91      0.92      0.91      3657
weighted avg       0.92      0.91      0.92      3657



#Adding Dropout to reduce overfitting

In [36]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 15, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 15, 40)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [38]:
### Finally Training
es=tf.keras.callbacks.EarlyStopping(patience=3,monitor='val_loss',restore_best_weights=True)
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20,batch_size=64,callbacks=[es])

Epoch 1/20
229/229 [==============================] - 6s 24ms/step - loss: 0.1906 - accuracy: 0.9210 - val_loss: 0.1903 - val_accuracy: 0.9119
Epoch 2/20
229/229 [==============================] - 5s 24ms/step - loss: 0.1274 - accuracy: 0.9495 - val_loss: 0.2293 - val_accuracy: 0.9171
Epoch 3/20
229/229 [==============================] - 5s 23ms/step - loss: 0.1010 - accuracy: 0.9614 - val_loss: 0.2542 - val_accuracy: 0.9122
Epoch 4/20
229/229 [==============================] - 5s 23ms/step - loss: 0.0790 - accuracy: 0.9715 - val_loss: 0.2323 - val_accuracy: 0.9125


#Performance Metrics And Accuracy


In [39]:
y_pred=model.predict_classes(X_test)
from sklearn.metrics import confusion_matrix,classification_report
print("Confusion Matrix is: \n",confusion_matrix(y_test,y_pred))
print("\n")
print("Classification Report is: \n",classification_report(y_test,y_pred))

Confusion Matrix is: 
 [[1848  234]
 [  88 1487]]


Classification Report is: 
               precision    recall  f1-score   support

           0       0.95      0.89      0.92      2082
           1       0.86      0.94      0.90      1575

    accuracy                           0.91      3657
   macro avg       0.91      0.92      0.91      3657
weighted avg       0.92      0.91      0.91      3657



In [40]:
submission=pd.read_csv('submit.csv')
submission['label']=model.predict_classes(test_final)
print(submission.shape)

(5200, 2)


In [41]:
submission.head()

,id,label
0,20800,0
1,20801,1
2,20802,1
3,20803,0
4,20804,1


In [42]:
submission['label'].value_counts()

1    2679
0    2521
Name: label, dtype: int64

In [43]:
submission.to_csv('fake_01.csv',index=False)